In [1]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
    PT = f.read()
    #PT = Plain text or raw text from book 
print("no. of char in this book:",len(PT))
print(PT[:100])

no. of char in this book: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


In [2]:
import re
# FT is reformed text 
FT = re.split(r'([,.:;?_!"()\']|--|\s)',PT) # it split each word characters(?/--= etc ) and whitespase( )
FT = [item.strip() for item in FT if item.strip()]# it removes the whitespace
print(FT[:30])
print(len(FT))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4690


In [3]:
allwords = sorted(set(FT))
print(len(allwords))

# see altho there is 4690 words but when u sort it by alphbetialy it only takes unique words as The and the is same.

1130


In [4]:
vocab = {token:integer for integer,token in enumerate(allwords)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

print(len(vocab))

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
1130


In [5]:
class STV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [6]:
tokenizer = STV1(vocab)

text = """ The ? " ()"""
ids = tokenizer.encode(text)
print(ids)

[93, 10, 1, 3, 4]


In [7]:
tokenizer = STV1(vocab)

ids=[93, 10,1, 3, 4]
text = tokenizer.decode(ids)
print(text)

The?"()


In [8]:
allwords = sorted(set(FT))
allwords.extend(["<|endoftext|>", "<|BOS|>","<|unk|>"])
vocab = {token:integer for integer,token in enumerate(allwords)}
print(len(vocab))

1133


In [9]:
for i, item in enumerate(list(vocab.items())[-3:]):
    print(item)

('<|endoftext|>', 1130)
('<|BOS|>', 1131)
('<|unk|>', 1132)


In [10]:
class STV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [11]:
tokenizer = STV2(vocab)

start = "<|BOS|>"
end = "<|endoftext|>"
text1 = "Hello, This is soham."
text11 = start + " " + text1 + " " + end
text2 = "In the sky is good?"
text22 = start + " " + text2 + " " + end

text = text11 + " " + text22

print(text)

<|BOS|> Hello, This is soham. <|endoftext|> <|BOS|> In the sky is good? <|endoftext|>


In [15]:
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text)))

[1131, 1132, 5, 97, 584, 1132, 7, 1130, 1131, 55, 988, 1132, 584, 500, 10, 1130]
<|BOS|> <|unk|>, This is <|unk|>. <|endoftext|> <|BOS|> In the <|unk|> is good? <|endoftext|>


### BYTE PAIR ENCODING (BPE)


In [27]:
import importlib
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

In [28]:
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [29]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.
